# DSCI 551 Project
Gordon Su, Lucas Huang  
Section 32414 (Afternoon)

Date range: 2020-12-02 to 2022-03-29

### All necessary imports

In [115]:
import json
import math
import numpy as np
import pandas as pd
import requests
import firebase_admin
from firebase_admin import db
from firebase_admin import credentials
import json
import os

### Import and preview vaccination data

In [116]:
vacc_df = pd.read_csv('./data/country_vaccinations.csv')
vacc_df['date'] = pd.to_datetime(vacc_df['date'])
vacc_df['year'] = vacc_df.apply(lambda row: row.date.year, axis=1)
vacc_df['month'] = vacc_df.apply(lambda row: row.date.month, axis=1)
vacc_df['day'] = vacc_df.apply(lambda row: row.date.day, axis=1)
vacc_df.head(3)

,country,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,daily_vaccinations_per_million,vaccines,source_name,source_website,year,month,day
0,Afghanistan,AFG,2021-02-22,0.0,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://covid19.who.int/,2021,2,22
1,Afghanistan,AFG,2021-02-23,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,34.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://covid19.who.int/,2021,2,23
2,Afghanistan,AFG,2021-02-24,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,34.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://covid19.who.int/,2021,2,24


### Import and preview PFE price data

In [3]:
pfe_prices_df = pd.read_csv('./data/PFE_historical_prices.csv')
pfe_prices_df['Date'] = pd.to_datetime(pfe_prices_df['Date'])
pfe_prices_df['year'] = pfe_prices_df.apply(lambda row: row.Date.year, axis=1)
pfe_prices_df['month'] = pfe_prices_df.apply(lambda row: row.Date.month, axis=1)
pfe_prices_df['day'] = pfe_prices_df.apply(lambda row: row.Date.day, axis=1)
pfe_prices_df.head(3)

,Date,Open,High,Low,Close,Adj Close,Volume,year,month,day
0,2020-12-02,40.470001,41.410000,40.299999,40.799999,38.354626,84347500,2020,12,2
1,2020-12-03,40.980000,41.029999,39.520000,40.090000,37.687180,68570100,2020,12,3
2,2020-12-04,39.889999,40.450001,39.889999,40.340000,37.922195,35368100,2020,12,4


### Iterate over vaccination data and upload to Firebase

In [5]:
all_countries = vacc_df['country'].unique()
all_years = [2020, 2021, 2022]
all_months = [month for month in range(1, 13)]
vacc_dict = {}

for country in all_countries:
    country_df = vacc_df.loc[vacc_df['country'] == country]
    country_dict = {}
    
    for year in all_years:
        year_df = vacc_df.loc[vacc_df['year'] == year]
        year_dict = {}
        
        for month in all_months:
            month_df = vacc_df.loc[vacc_df['month'] == month]
            month_dict = dict(month_df[['day', 'daily_vaccinations']].values)
            month_dict = {int(k):v for k,v in month_dict.items()}
            
            year_dict[month] = month_dict
        country_dict[year] = year_dict
    vacc_dict[country] = country_dict
    
url = 'https://dsci-551-3cb2b-default-rtdb.firebaseio.com/vaccination_data.json'
result = requests.put(url, data=json.dumps(vacc_dict))
print(result)

<Response [401]>


In [109]:
put_file('PFE_historical_prices.csv', '/test/', 10)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

### Iterate over PFE price data and upload to Firebase

In [8]:
pfe_dict = {}

for year in all_years:
    year_df = pfe_prices_df.loc[pfe_prices_df['year'] == year]
    year_dict = {}

    for month in all_months:
        month_df = pfe_prices_df.loc[pfe_prices_df['month'] == month]
        month_dict = dict(month_df[['day', 'Close']].values)
        month_dict = {int(k):v for k,v in month_dict.items()}
        
        year_dict[month] = month_dict
    pfe_dict[year] = year_dict

url = 'https://dsci-551-3cb2b-default-rtdb.firebaseio.com/pfe_prices_data.json'
result = requests.put(url, data=json.dumps(pfe_dict))
print(result)

<Response [200]>


### EDFS Implementations

In [125]:
cred = credentials.Certificate('./dsci-551-3cb2b-firebase-adminsdk-7ajia-c0dffd1d3c.json')
databaseURL = 'https://dsci-551-3cb2b-default-rtdb.firebaseio.com/'
default_app = firebase_admin.initialize_app(cred, {
    'databaseURL': databaseURL
})
ref = db.reference('/')

ValueError: The default Firebase app already exists. This means you called initialize_app() more than once without providing an app name as the second argument. In most cases you only need to call initialize_app() once. But if you do want to initialize multiple apps, pass a second argument to initialize_app() to give each app a unique name.

In [122]:
# help
def edfs_help():
    print('''\033[1m\033[4mFunctions:\033[0m\033[0m
    \033[1mremove_file(path)\033[0m \t\t\t Usage example: remove_file('/test/test.json')
    \033[1mremove_directory(path)\033[0m \t\t Usage example: remove_directory('/test/')
    \033[1mcat_file(path)\033[0m \t\t\t Usage example: cat_file('/test/test.json')
    \033[1mlist_contents(path)\033[0m \t\t Usage example: list_contents('/test/')
    \033[1mmake_directory(path)\033[0m \t\t Usage example: make_directory('/test/')
    \033[1mput_file(file, path)\033[0m \t\t Usage example: put_file('test.json', '/test/')''')

# rm
def remove_file(path):
    splitPath = os.path.splitext(path)
    
    # Remove file metadata
    metadata_ref = db.reference(splitPath[0] + splitPath[1].replace('.', ' '))
    children = metadata_ref.get()
    if children == None:
        print('File does not exist')
        return
    for key, value in children.items():
        metadata_ref.child(key).set({})
        
    # Remove file partition contents
    partition_contents_ref = db.reference('/partitions' + splitPath[0] + splitPath[1].replace('.', ' '))
    children = partition_contents_ref.get()
    if children != None:
        for key, value in children.items():
            partition_contents_ref.child(key).set({})
        
# rm directory
def remove_directory(path):
    # Remove metadata directory
    metadata_ref = db.reference(path)
    children = metadata_ref.get()
    if children == None:
        print('Directory does not exist')
        return
    for key, value in children.items():
        metadata_ref.child(key).set({})
        
    # Remove partition contents directory
    partition_contents_ref = db.reference('/partitions' + path)
    children = partition_contents_ref.get()
    if children != None:
        for key, value in children.items():
            partition_contents_ref.child(key).set({})

# cat
def cat_file(path):
    splitPath = os.path.splitext(path)
    metadata_ref = db.reference(splitPath[0] + splitPath[1].replace('.', ' '))
    contents = metadata_ref.get()
    if contents == None:
        print('File does not exist')
        return
    for key, value in contents.items():
        partition_ref = db.reference(value)
        partition_contents = partition_ref.get()
        print(json.dumps(partition_contents, indent=1))

# ls
def list_contents(path):
    ref = db.reference(path)
    contents = ref.get()
    if contents == None:
        print('Directory does not exist')
        return
    for key, val in contents.items():
        print(key)   
        
# mkdir
def make_directory(path):
    # Set new path for metadata
    metadata_ref = db.reference(path)
    metadata_ref.set({
        'New Path':
        {
            'New_Path': True
        }
    })
    
    # Set new path for partition contents
    partition_contents_ref = db.reference('/partitions' + path)
    partition_contents_ref.set({
        'New Path':
        {
            'New_Path': True
        }
    })
    
# put
def put_file(file, path, num_partitions=1):
    dir_ref = db.reference(path)
    query = dir_ref.child('New Path').get()
    if query != None:
        dir_ref.child('New Path').set({})
        
    name_of_file = file.replace('.', ' ')
    file_ref = db.reference(path + name_of_file)
    
    dir_partition_contents_ref = db.reference('/partitions' + path)
    query = dir_partition_contents_ref.child('New Path').get()
    if query != None:
        dir_partition_contents_ref.child('New Path').set({})
        
    with open('./data/' + file, 'r') as f:
        contents = json.load(f)
        contents_df = pd.DataFrame(contents.items())
        total_rows = len(contents_df)
        num_rows_per_partition = math.ceil(total_rows / num_partitions)
        partition_metadata = {}
        
        for partition in range(1, num_partitions + 1):
            # Index of first row of partition (inclusive)
            start_row = (partition - 1) * num_rows_per_partition
            # Index of last row of partition (exclusive)
            end_row = min(total_rows, partition * num_rows_per_partition)
            
            partition_name = 'p' + str(partition)
            partition_path = '/partitions' + path + name_of_file + '/' + partition_name
            partition_ref = db.reference(partition_path)
            partition_dict = {}
            
            # Store metadata
            partition_metadata[partition_name] = partition_path
            
            # Store actual contents in partition
            for row in range(start_row, end_row):
                row_key = contents_df.iloc[row][0]
                row_value = contents_df.iloc[row][1]
                partition_dict[row_key] = row_value
            partition_ref.set(partition_dict)
                    
            if end_row == total_rows:
                break
                
        file_ref.set(partition_metadata)
        
def getPartitionLocations(file_path):
    splitPath = os.path.splitext(file_path)
    
    metadata_ref = db.reference(splitPath[0] + splitPath[1].replace('.', ' '))
    children = metadata_ref.get()
    if children == None:
        print('File does not exist')
        return
    for key, value in children.items():
        print(value)
    
def readPartition(file_path, partition_num):
    partition_name = 'p' + str(partition_num)
    splitPath = os.path.splitext(file_path)
    
    partition_content_ref = db.reference('/partitions' + splitPath[0] + splitPath[1].replace('.', ' '))
    children = partition_content_ref.get()
    
    if children == None:
        print('File does not exist')
        return
    for key, value in children.items():
        if key == partition_name:
            print(value)
            return
    print('Partition does not exist')

In [118]:
getPartitionLocations('/test/test.json')

/partitions/test/test json/p1
/partitions/test/test json/p2


In [124]:
readPartition('/test/test.json', 2)

{'Person3': {'description': 'Eligendi voluptate in nostrum odit. Et voluptas mollitia suscipit. Optio quas ut voluptas.', 'email': 'patience67@gmail.com', 'name': 'Mr. Daryl Leffler Jr.', 'phone': '+1-551-867-7600'}}


In [52]:
put_file('test.json', '/test/', 2)

In [53]:
remove_file('/test/test.json')

In [29]:
edfs_help()

Functions:
    remove_file(path) 			 Usage example: remove_file('/test/test.json')
    remove_directory(path) 		 Usage example: remove_directory('/test/')
    cat_file(path) 			 Usage example: cat_file('/test/test.json')
    list_contents(path) 		 Usage example: list_contents('/test/')
    make_directory(path) 		 Usage example: make_directory('/test/')
    put_file(file, path) 		 Usage example: put_file('test.json', '/test/')


In [78]:
make_directory('/test/')

In [100]:
put_file('test.json', '/test/', 2)

In [99]:
remove_file('/test/test.json')

In [77]:
remove_directory('/test/')

In [76]:
list_contents('/test/')

test json


In [107]:
cat_file('/test/test.json')

{
 "Person1": {
  "description": "Dolor a sit illum vel corrupti est quia. Dolorem repudiandae molestiae qui ut. Repellendus in mollitia error repellat necessitatibus molestiae.",
  "email": "cale83@hotmail.com",
  "name": "Lorenza Nienow",
  "phone": "1-786-677-7753"
 },
 "Person2": {
  "description": "Amet veniam dolorum dolor sit voluptas. Voluptatem sit amet et provident. Culpa soluta nobis voluptatibus blanditiis animi veritatis officia.",
  "email": "dietrich.janet@gmail.com",
  "name": "Jack Koch PhD",
  "phone": "947.729.3179"
 }
}
{
 "Person3": {
  "description": "Eligendi voluptate in nostrum odit. Et voluptas mollitia suscipit. Optio quas ut voluptas.",
  "email": "patience67@gmail.com",
  "name": "Mr. Daryl Leffler Jr.",
  "phone": "+1-551-867-7600"
 }
}


In [152]:
list_contents('/dummy/')

Directory does not exist


In [153]:
cat_file('/dummy/dummy.json')

File does not exist


In [154]:
make_directory('/test2/')

In [155]:
put_file('test.json', '/test2/')

In [159]:
remove_file('/test/test.json')

In [157]:
remove_file('/test2/test.json')